In [0]:
# Dataset
# DataFrame
# Sql table & view

# Spark : 트랜스포메이션의 처리과정을 정의하는 분산 프로그래밍 모델. 다수의 트랜스포메이션은 DAG로 표현되는 명령을 만들고, 액션은 하나의 잡을 클러스터에서 실행하기 위해 스테이지와 태스크로 나누며 DAG처리 프로세스를 실행함. 트랜스포메이션과 액션으로 다루는 논리적 구조가 DataFrame과 Dataset.

In [0]:
# DataFrame과 Dataset은 결과를 생성하기 위해 어떤 데이터에 어떤 연산을 적용해야 하는지 정의하는 지연 연산의 실행 계획이며 불변함.
# DataFrame에 액션을 호출하면 스파크는 트랜스포메이션을 실제로 실행하고 결과를 반환함.
# 테이블과 뷰는 DataFrame과 기본적으로 동일 다만, 코드 대신 SQL을 쓰는 차이만 존재
# 카탈리스트 : 실행 계획 수립과 처리에 사용하는 자체 데이터 타입의 정보를 가지고 있으며, 여러 언어 API와 직접 매핑됨


df = spark.range(500).toDF("number")
df.select(df["number"] + 10)

Out[2]: DataFrame[(number + 10): bigint]

In [0]:
# 비 타입형 : DataFrame - 데이터 타입의 일치 여부를 런타임에서 확인 - 파이썬, R
# 타입형 : Dataset - 데이터 타입의 일치 여부를 컴파일에서 확인 - 스칼라와 자바에서만 지원


# 실행과정 
1. Dataset/DataFrame/SQL을 이용해 코드 작성
2. 논리적 실행계획으로 변경
3. 물리적 실행계획으로 변경(이때 추가적인 최적화 실행[카탈리스트 옵티마이저])
4. 물리적 실행계획(RDD) 실행